<a href="https://colab.research.google.com/github/subjayap-git/Learndevops/blob/master/webapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import joblib
import pandas as pd
from flask import Flask, request, jsonify, render_template
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

app = Flask(__name__)

# Load dataset
file_path = "PCOS_SURVEY_SPREADSHEET.csv"
df = pd.read_csv(file_path)

# Clean column names
df.columns = [
    "Age", "Weight", "Height", "Blood_Group", "Cycle_Interval",
    "Recent_Weight_Gain", "Excess_Hair_Growth", "Skin_Darkening",
    "Hair_Loss", "Acne", "Fast_Food", "Exercise", "PCOS_Diagnosed",
    "Mood_Swings", "Regular_Periods", "Period_Duration"
]

# Selecting features and target variable
X = df.drop(columns=["PCOS_Diagnosed", "Blood_Group"])
y = df["PCOS_Diagnosed"]

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardizing numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Training the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Save model and scaler
joblib.dump(model, "pcos_model.pkl")
joblib.dump(scaler, "scaler.pkl")

# Evaluating the model
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Load model and scaler
model = joblib.load("pcos_model.pkl")
scaler = joblib.load("scaler.pkl")

@app.route("/")
def home():
    return '''
    <html>
    <head>
        <title>PCOS Prediction</title>
        <style>
            body { background-color: black; color: orange; font-family: Arial, sans-serif; text-align: center; }
            h1 { color: orange; }
            form { margin: 20px auto; width: 50%; }
            input, button { padding: 10px; margin: 5px; }
            button { background-color: orange; color: black; border: none; cursor: pointer; }
            button:hover { background-color: #ff8800; }
        </style>
    </head>
    <body>
        <h1>PCOS Prediction</h1>
        <form action="/predict" method="post">
            Age: <input type="number" name="age" required><br>
            Weight (kg): <input type="number" name="weight" required><br>
            Height (cm): <input type="number" name="height" required><br>
            Cycle Interval (months): <input type="number" name="cycle_interval" required><br>
            Recent Weight Gain: <input type="number" name="weight_gain" min="0" max="1" required><br>
            Excess Hair Growth: <input type="number" name="hair_growth" min="0" max="1" required><br>
            Skin Darkening: <input type="number" name="skin_darkening" min="0" max="1" required><br>
            Hair Loss: <input type="number" name="hair_loss" min="0" max="1" required><br>
            Acne: <input type="number" name="acne" min="0" max="1" required><br>
            Fast Food Consumption: <input type="number" name="fast_food" min="0" max="1" required><br>
            Exercise Regularly: <input type="number" name="exercise" min="0" max="1" required><br>
            Mood Swings: <input type="number" name="mood_swings" min="0" max="1" required><br>
            Regular Periods: <input type="number" name="regular_periods" min="0" max="1" required><br>
            Period Duration (days): <input type="number" name="period_duration" required><br>
            <button type="submit">Predict</button>
        </form>
    </body>
    </html>
    '''

@app.route("/predict", methods=["POST"])
def predict_pcos():
    data = request.form
    try:
        input_data = np.array([[
            float(data["age"]), float(data["weight"]), float(data["height"]), float(data["cycle_interval"]),
            int(data["weight_gain"]), int(data["hair_growth"]), int(data["skin_darkening"]),
            int(data["hair_loss"]), int(data["acne"]), int(data["fast_food"]), int(data["exercise"]),
            int(data["mood_swings"]), int(data["regular_periods"]), float(data["period_duration"])
        ]])

        # Scale input
        input_scaled = scaler.transform(input_data)

        # Get probability
        prob_pcos = model.predict_proba(input_scaled)[:, 1][0]

        # Categorize probability
        if prob_pcos < 0.3:
            result = "No PCOS detected - Maintain a good lifestyle"
        elif prob_pcos < 0.7:
            result = "May have PCOS - Can visit a doctor"
        else:
            result = "More possibility of PCOS - Must visit a doctor"

        return f'<body style="background-color:black; color:orange; text-align:center;"><h1>PCOS Prediction</h1><p>{result}</p><p>Probability: {prob_pcos:.2f}</p></body>'
    except Exception as e:
        return f'<body style="background-color:black; color:orange; text-align:center;"><h1>Error</h1><p>{str(e)}</p></body>'

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

Model Accuracy: 0.83
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
